### Выбор значимых признаков при помощи Логистической регрессии

Чтобы понять, действительно ли есть зависимость между признаком и целевой переменной нужно посмотреть на отношение коэффициента регрессии к погрешности этого коэффициента. Это отношение будет t-статистикой для нулевой гипотезы, состоящей в том, что между целевой переменной и признаком нет зависимости.

Если отношение достаточно большое, то принимается гипотеза о наличии зависимости. Однако следует относиться к таким оценкам с осторожностью. Если признак не является независимым, то у соответствующего коэффициента может быть любое значение.

In [2]:
import category_encoders as ce
import numpy as np
import pandas as pd
from scipy.stats import t
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [3]:
url = \
    'https://raw.githubusercontent.com/TaranovEV/Datasets/master/australian.dat.txt'

df = pd.read_csv(
    url, 
    sep=' ', 
    header=None)
df.columns = ['A' + str(i) for i in range(1, 15)] + ['loan']

**Смотрим на полноту данных(отсутствие пропусков)**

In [4]:
na_values = df.isnull().sum()
print('Сумма значений пропусков из',len(df),'значений:',na_values.sum())

Сумма значений пропусков из 690 значений: 0


In [5]:
# разбиваем выборку:
# y - целевая переменная loan
# X - матрица признаков
X = df.drop('loan', axis=1)
y = df['loan']
# далее делим в отношении 80/20:
# X_train, y_train - тренировочная выборка
# X_test, y_test -тестовая выборка
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Применим среднечастотное кодирование категориальных признаков

In [6]:
categorical = ['A' + str(i) for i in [1, 4, 5, 6, 8, 9, 11, 12]]
numerical = ['A' + str(i) for i in [2, 3, 7, 10, 13, 14]]

In [7]:
transformer = ce.woe.WOEEncoder(cols=categorical)
X_train_woe = transformer.fit_transform(X_train, y_train)
X_test_woe = transformer.transform(X_test)

Матрица информации для логистической регрессии

$$\bigl[\mathcal{I}(w) \bigr]_{i, j}
 = X^T V X,$$
 
Дисперсия оценок параметров получается так:

$$Var(\hat{w}) = \text{diag}(\mathcal{I}^{-1}(\hat{w}))$$

>Единицы в матрице $X$ введены, так как первый элемент $w$ &mdash; смещение. $\text{diag()}$ возвращает диагональ матрицы.

In [8]:
# создадим функцию возвращающую 
def get_coef_std(clf, X):
    #получим список вероятности принадлежности y 
    Z =clf.predict_proba(X)
    #создаем смещение - единичный вектор размерности Х
    Y = np.ones((X.shape[0], 1))
        
    #объединяем с вектором Х
    X_new = np.concatenate((Y,X),axis=1)
    
    #в вектор V заполним диагональ произведением вероятностей y*(1-y)
    V = np.diagflat(np.prod(Z, axis=1))
    #выполним произведение для получения матрицы информации
    I = X_new.T @ V @ X_new
    #вычисляем обратную матрицу
    I_1 = np.linalg.inv(I)
    
    #дисперсия оценок будет диагональю матрицы
    var = np.diag(I_1)
    
    #стандартное отклонение
    se = np.sqrt(var)
    #возвращаем значения 
    return se

In [9]:
# обучим модель
log_reg = LogisticRegression(penalty='l2',solver='liblinear',fit_intercept=True)
log_reg.fit(X_train_woe, y_train)
pass

In [10]:
# вычислим отклонение
std = get_coef_std(log_reg, X_train_woe)

In [15]:
def build_table(log_model,X_test):
    global coef_table
# создадим DataFrame содержащий следующую информацию:
# смещение признака
    bias_coef = np.r_[log_model.intercept_, log_model.coef_[0]]
# t статистику 
    t_statistics = bias_coef / std
# p-уровень значимости
    p_values = 1 - t.cdf(
        np.abs(t_statistics),
        X_test.shape[0] - X_test.shape[1] - 1
    )
# значения будем выводить в формате 3 знака после запятой
    pd.options.display.float_format = '{:,.3f}'.format
    coef_table = pd.DataFrame(
        {
            'Feature': ['intercept'] + list(X_test.columns),
            'coef': bias_coef,
            'coef_std': std,
            't': t_statistics,
            'p-value': p_values
        }
    )

    return coef_table
build_table(log_reg,X_test_woe)

,Feature,coef,coef_std,t,p-value
0,intercept,-0.044,0.468,-0.093,0.463
1,A1,0.012,3.758,0.003,0.499
2,A2,-0.012,0.013,-0.963,0.169
3,A3,0.002,0.032,0.056,0.478
4,A4,0.829,0.356,2.331,0.011
5,A5,0.970,0.212,4.567,0.000
6,A6,-0.140,0.314,-0.445,0.329
7,A7,0.058,0.054,1.065,0.144
8,A8,0.856,0.087,9.824,0.000
9,A9,0.154,0.200,0.770,0.221


Проведем небольшое исследование -построим матрицу корреляции признаков и посмотрим какик признаки имеют наибольшее значение

In [16]:
matrix_correlation = df.corr().round(3)
#отберем значения больше 0,5
most_importance = matrix_correlation[matrix_correlation>abs(0.5)]
most_importance

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,loan
A1,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
A2,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
A3,nan,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
A4,nan,nan,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
A5,nan,nan,nan,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
A6,nan,nan,nan,nan,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
A7,nan,nan,nan,nan,nan,nan,1.000,nan,nan,nan,nan,nan,nan,nan,nan
A8,nan,nan,nan,nan,nan,nan,nan,1.000,nan,nan,nan,nan,nan,nan,0.720
A9,nan,nan,nan,nan,nan,nan,nan,nan,1.000,0.571,nan,nan,nan,nan,nan
A10,nan,nan,nan,nan,nan,nan,nan,nan,0.571,1.000,nan,nan,nan,nan,nan


In [17]:
print('полученнное значение',most_importance.loc['A8','loan'], 'для A8 \
подтверждает наличие зависимости признака целевой переменной,также как и \
самого большого значения отношения',coef_table.iloc[8,3].round(3))

полученнное значение 0.72 для A8 подтверждает наличие зависимости признака целевой переменной,также как и самого большого значения отношения 9.824


Теперь посмотрим как сработает данный подход когда признаки станут зависимы между собой.
При использовании ohe-кодирования новые признаки получаются линейно зависимыми, а коэффициенты регрессии при линейно-зависимых признаках, если модель обучалась без регуляризации, могут быть любыми и погрешность становится бесконечно большой.  

In [18]:
#проведем преобразования категориальных признаков
X_ohe = pd.get_dummies(X,prefix=categorical,columns =categorical)
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.2, random_state=42)

#обучаем модель
Ohe_log_reg = LogisticRegression(solver='liblinear',fit_intercept=True)
Ohe_log_reg.fit(X_train, y_train)
std = get_coef_std(Ohe_log_reg, X_train)
build_table(Ohe_log_reg,X_test)

,Feature,coef,coef_std,t,p-value
0,intercept,-0.097,"6,389,823.711",-0.000,0.500
1,A2,-0.008,0.014,-0.621,0.268
2,A3,0.009,0.032,0.268,0.395
3,A7,0.064,0.056,1.147,0.127
4,A10,0.135,0.064,2.100,0.019
5,A13,-0.002,0.001,-1.703,0.046
6,A14,0.001,0.000,2.956,0.002
7,A1_0,-0.012,"2,770,268.738",-0.000,0.500
8,A1_1,-0.086,nan,nan,nan
9,A4_1,-0.563,"9,488,340.491",-0.000,0.500
